In [46]:
import dash
from dash import Dash, dcc, html
from dash.dependencies import Input, Output
import pandas as pd
import numpy as np
import re 
import requests

In [47]:
from _NBAModelSchedule import get_todays_games
from _NBAModelTeamsDivisions import process_league_standings

In [48]:
%run _NBAModelTeamsDivisions.py

Number of teams fetched: 30


# Schedule

In [49]:
nba_schedule_df = pd.read_csv('nbaSchedule24.csv')

In [50]:
def get_todays_games(df, today):
    todays_games=df[df["Game Date"] == today]
    return todays_games

In [51]:
todays_games = get_todays_games(nba_schedule_df, '2024-10-22')

In [52]:
todays_games

,Game Date,Game ID,Game Time,Arena,Arena City,Home Team Abbreviation,Home Conference,Home Division,Visiting Team Abbreviation,Visiting Conference,Visiting Division,Divisional Game,Conference Game
0,2024-10-22,22400061,7:30 pm ET,TD Garden,Boston,BOS,East,Atlantic,NYK,East,Atlantic,Yes,Yes
1,2024-10-22,22400062,10:00 pm ET,Crypto.com Arena,Los Angeles,LAL,West,Pacific,MIN,West,Northwest,No,Yes


# Standings

In [53]:
# Assuming process_league_standings is already defined
standings_df = process_league_standings('2024-25', 'Regular Season')

# Teams

In [60]:
from _NBAModelTeamsDivisions import process_league_standings, season_standings

In [61]:
from _NBAModelTeamsDivisions import NBATeamData

ImportError: cannot import name 'NBATeamData' from '_NBAModelTeamsDivisions' (C:\Users\justi\Documents\__ballAnalytics\bball\NBAModel\_NBAModelTeamsDivisions.py)

In [ ]:
# Create an instance of the class
nba_data = NBATeamData()

# Fetch the teams and print the DataFrame
teams_df = nba_data.fetch_teams()
print(teams_df.head())

# Use another method from the class
team_info = nba_data.get_team_by_abbreviation('LAL')
print(team_info)


# Create App

In [11]:
app = Dash(__name__, suppress_callback_exceptions=True)

app.layout = html.Div([
    dcc.Tabs([
        dcc.Tab(label='Today\'s Matchups', children=[
            html.Div(id='todays-matchups')
        ]),
        dcc.Tab(label='Standings', children=[
            html.Div(id='standings')
        ]),
        dcc.Tab(label='Matchup Analyzer', children=[
            html.Div(id='matchup-analyzer')
        ]),
    ])
])

In [12]:
# Callback to render today's matchups
@app.callback(
    Output('todays-matchups', 'children'),
    Input('tabs', 'value')
)
def render_todays_matchups(tab):
    today_date = '2025-01-01'  # Replace with dynamic date as needed
    todays_games_df = get_todays_games(nba_schedule_df, today_date)
    
    return html.Table([
        html.Tr([html.Th(col) for col in todays_games_df.columns])] +
        [html.Tr([html.Td(todays_games_df.iloc[i][col]) for col in todays_games_df.columns])
         for i in range(len(todays_games_df))]
    )

# Callback to render standings
@app.callback(
    Output('standings', 'children'),
    Input('tabs', 'value')
)
def render_standings(tab):
    return html.Table([
        html.Tr([html.Th(col) for col in standings_df.columns])] +
        [html.Tr([html.Td(standings_df.iloc[i][col]) for col in standings_df.columns])
         for i in range(len(standings_df))]
    )

# Callback for the Matchup Analyzer (placeholder)
@app.callback(
    Output('matchup-analyzer', 'children'),
    Input('tabs', 'value')
)
def render_matchup_analyzer(tab):
    return html.Div([
        html.H3('Matchup Analyzer will be developed here.')
    ])

In [13]:
if __name__ == '__main__':
    app.run_server(debug=True)